In [21]:
# https://learn.microsoft.com/zh-cn/archive/msdn-magazine/2019/march/test-run-neural-regression-using-pytorch
# https://learn.microsoft.com/zh-cn/archive/msdn-magazine/2019/march/code-downloads-for-march-2019-msdn-magazine

In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [23]:
def accuracy(model, data_x, data_y, pct_close):
    n_items = len(data_y)
    X = torch.tensor(data_x, dtype=torch.float32)  # 2-d Tensor
    Y = torch.tensor(data_y, dtype=torch.float32)  # actual as 1-d Tensor

    oupt = model(X)  # all predicted as 2-d Tensor
    pred = oupt.view(n_items)  # all predicted as 1-d

    n_correct = torch.sum((torch.abs(pred - Y) < torch.abs(pct_close * Y)))
    result = (n_correct.item() * 100.0 / n_items)  # scalar
    return result

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = nn.Linear(13, 20)  # 13-(20-20-10)-1
        self.hid2 = nn.Linear(20, 20)
        self.hid3 = nn.Linear(20, 10)
        self.oupt = nn.Linear(10, 1)

        nn.init.xavier_uniform_(self.hid1.weight)  # glorot
        nn.init.zeros_(self.hid1.bias)
        nn.init.xavier_uniform_(self.hid2.weight)
        nn.init.zeros_(self.hid2.bias)
        nn.init.xavier_uniform_(self.hid3.weight)
        nn.init.zeros_(self.hid3.bias)
        nn.init.xavier_uniform_(self.oupt.weight)
        nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = torch.tanh(self.hid1(x))  # or nn.Tanh()
        z = torch.tanh(self.hid2(z))
        z = torch.tanh(self.hid3(z))
        z = self.oupt(z)  # no activation, aka Identity()
        return z

In [25]:
def main():
    # 0. get started
    print("Boston regression using PyTorch 2.0 \n")
    torch.manual_seed(1)
    rng = np.random.default_rng(1)

    # 1. load data
    print("Loading Boston data into memory")
    train_file = "./data/boston_train.txt"
    test_file = "./data/boston_test.txt"

    train_x = np.loadtxt(
        train_file,
        delimiter="\t",
        usecols=range(0, 13),
        dtype=np.float32
    )
    train_y = np.loadtxt(
        train_file,
        delimiter="\t",
        usecols=[13],
        dtype=np.float32
    )
    test_x = np.loadtxt(
        test_file,
        delimiter="\t",
        usecols=range(0, 13),
        dtype=np.float32
    )
    test_y = np.loadtxt(
        test_file,
        delimiter="\t",
        usecols=[13],
        dtype=np.float32
    )

    # 2. create model
    print("Creating 13-(20-20-10)-1 DNN regression model\n")
    net = Net()  # all work done above

    # 3. train model
    net.train()  # set training mode
    bat_size = 10
    loss_func = nn.MSELoss()  # mean squared error
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    n_items = len(train_x)
    batches_per_epoch = n_items // bat_size
    max_batches = 1000 * batches_per_epoch

    print("Starting training")
    for b in range(max_batches):
        curr_bat = rng.choice(n_items, bat_size, replace=False)
        X = torch.tensor(train_x[curr_bat], dtype=torch.float32)
        Y = torch.tensor(train_y[curr_bat], dtype=torch.float32).view(bat_size, 1)
        optimizer.zero_grad()
        oupt = net(X)

        loss_obj = loss_func(oupt, Y)
        loss_obj.backward()  # compute gradients
        optimizer.step()  # update weights and biases

        if b % (max_batches // 10) == 0:
            print(f"batch = {b:6d}", end="")
            print(f"\t batch loss = {loss_obj.item():7.4f}", end="")
            net.eval()
            acc = accuracy(net, train_x, train_y, 0.15)
            net.train()
            print(f"\t accuracy = {acc:.2f}%")
    print("Training complete\n")

    # 4. evaluate model
    net.eval()  # set eval mode
    acc = accuracy(net, test_x, test_y, 0.15)
    print(f"Accuracy on test data = {acc:.2f}%")

    # 5. save model
    model_path = "./boston_model.pth"
    torch.save(net.state_dict(), model_path)
    print(f"Model saved to {model_path}\n")

    # 6. use model - load model from file
    loaded_net = Net()
    loaded_net.load_state_dict(torch.load(model_path))
    loaded_net.eval()
    print(f"Model loaded from {model_path}\n")

    raw_inpt = np.array(
        [
            [
                0.09266, 34, 6.09, 0, 0.433,
                6.495, 18.4, 5.4917, 7, 329,
                16.1, 383.61, 8.67
            ]
        ],
        dtype=np.float32
    )

    norm_inpt = np.array(
        [
            [
                0.000970, 0.340000, 0.198148, -1, 0.098765,
                0.562177, 0.159629, 0.396666, 0.260870, 0.270992,
                0.372340, 0.966488, 0.191501
            ]
        ],
        dtype=np.float32
    )

    X = torch.tensor(norm_inpt, dtype=torch.float32)
    y = loaded_net(X)
    print("For a town with raw input values: ")
    for (idx, val) in enumerate(raw_inpt[0]):
        if idx % 5 == 0:
            print("")
        print(f"{val:11.6f} ", end="")
    print(f"\n\nPredicted median house price = ${y.item() * 10000:.2f}")

if __name__ == "__main__":

    main()


Boston regression using PyTorch 2.0 

Loading Boston data into memory
Creating 13-(20-20-10)-1 DNN regression model

Starting training
batch =      0	 batch loss =  3.6017	 accuracy = 0.50%
batch =   4000	 batch loss =  0.0399	 accuracy = 74.75%
batch =   8000	 batch loss =  0.0406	 accuracy = 75.50%
batch =  12000	 batch loss =  0.0298	 accuracy = 77.97%
batch =  16000	 batch loss =  0.0321	 accuracy = 79.21%
batch =  20000	 batch loss =  0.0800	 accuracy = 79.70%
batch =  24000	 batch loss =  0.0859	 accuracy = 83.91%
batch =  28000	 batch loss =  0.0936	 accuracy = 84.16%
batch =  32000	 batch loss =  0.0486	 accuracy = 85.40%
batch =  36000	 batch loss =  0.0092	 accuracy = 82.92%
Training complete

Accuracy on test data = 79.41%
Model saved to ./boston_model.pth

Model loaded from ./boston_model.pth

For a town with raw input values: 

   0.092660   34.000000    6.090000    0.000000    0.433000 
   6.495000   18.400000    5.491700    7.000000  329.000000 
  16.100000  383.609985  